# 🏗️ RAG Engine Architecture & Design Patterns

> **Educational Notebook 25**: Deep dive into the architecture, design patterns, and engineering principles behind the RAG Engine

---

## 📋 Learning Objectives

By the end of this notebook, you will:

1. Understand the Clean Architecture principles implemented in the RAG Engine
2. Learn about the Ports & Adapters pattern and its implementation
3. Explore the dependency injection system and container design
4. Analyze the domain-driven design approach
5. Master the patterns for extensibility and maintainability

## 🎯 The Architecture Philosophy

The RAG Engine Mini implements Clean Architecture with a focus on maintainability, testability, and extensibility. This architecture ensures that business rules are independent of frameworks, UI, database, or external agencies.

## 🛠️ Setting Up Our Environment

First, let's set up our environment and import the necessary libraries:

In [1]:
import os
import sys
from pathlib import Path
import ast
import inspect
from typing import List, Dict, Any
import pprint

# Add project root to path
current = Path().resolve()
repo_root = None
for parent in [current, *current.parents]:
    if (parent / "src").exists() and (parent / "notebooks").exists():
        repo_root = parent
        break

if repo_root is None:
    raise RuntimeError("Could not locate rag-engine-mini root for imports")

sys.path.insert(0, str(repo_root))

print(f"Repository root: {repo_root}")
print("Environment setup complete!")

## 🏗️ Clean Architecture Overview

Let's examine the Clean Architecture layers implemented in the RAG Engine:

In [2]:
# Define the Clean Architecture layers
clean_architecture_layers = {
    "Domain Layer": {
        "purpose": "Contains business entities, rules, and interfaces",
        "location": "src/domain/",
        "contents": ["entities.py", "errors.py"],
        "characteristics": [
            "Pure business logic",
            "No external dependencies",
            "Independent of frameworks"
        ]
    },
    "Application Layer": {
        "purpose": "Contains use cases and business services",
        "location": "src/application/",
        "subdirs": ["ports/", "services/", "use_cases/"],
        "characteristics": [
            "Orchestrates business operations",
            "Depends on Domain layer",
            "Defines interfaces for external services"
        ]
    },
    "Adapters Layer": {
        "purpose": "Implements external service interfaces",
        "location": "src/adapters/",
        "subdirs": ["llm/", "embeddings/", "vector/", "persistence/", "cache/"],
        "characteristics": [
            "Implements application ports",
            "Depends on Application layer",
            "Integrates with external services"
        ]
    },
    "Interface Layer": {
        "purpose": "APIs, UI, and external interfaces",
        "location": "src/api/",
        "subdirs": ["v1/"],
        "characteristics": [
            "Exposes functionality to external world",
            "Depends on Application layer",
            "Thin controllers"
        ]
    }
}

# Display the architecture
print("Clean Architecture Layers in RAG Engine:")
print("=" * 70)
for layer, details in clean_architecture_layers.items():
    print(f"\n{layer}:")
    print(f"  Purpose: {details['purpose']}")
    print(f"  Location: {details['location']}")
    
    if 'contents' in details:
        print(f"  Contents: {', '.join(details['contents'])}")
    if 'subdirs' in details:
        print(f"  Subdirectories: {', '.join(details['subdirs'])}")
    
    print("  Characteristics:")
    for char in details['characteristics']:
        print(f"    • {char}")

## 🔌 Ports & Adapters Pattern

Let's explore the Ports & Adapters pattern implementation in the RAG Engine:

In [3]:
# Analyze the ports in the application layer
import importlib
import pkgutil

def find_ports_in_application():
    """Find all port interfaces in the application layer"""
    ports_info = []
    
    # Import the ports module
    try:
        from src.application.ports import llm_port
        from src.application.ports import embeddings_port
        from src.application.ports import vector_store
        from src.application.ports import document_repository
        from src.application.ports import reranker_port
        
        ports_info.extend([
            {"name": "LLMPort", "module": llm_port, "description": "Interface for LLM providers"},
            {"name": "EmbeddingsPort", "module": embeddings_port, "description": "Interface for embedding providers"},
            {"name": "VectorStorePort", "module": vector_store, "description": "Interface for vector storage"},
            {"name": "DocumentRepoPort", "module": document_repository, "description": "Interface for document persistence"},
            {"name": "RerankerPort", "module": reranker_port, "description": "Interface for re-ranking services"}
        ])
    except ImportError as e:
        print(f"Import error: {e}")
    
    return ports_info

# Display port information
ports = find_ports_in_application()

print("Ports in RAG Engine (Application Layer Interfaces):")
print("=" * 70)
for port in ports:
    print(f"\n{port['name']}: {port['description']}")
    print(f"  Module: {port['module'].__name__}")
    
    # Try to get the actual class definition
    try:
        port_class = getattr(port['module'], port['name'])
        sig = inspect.signature(port_class.__init__) if hasattr(port_class, '__init__') else 'No __init__ signature'
        print(f"  Signature: {sig}")
    except AttributeError:
        print(f"  Note: Could not retrieve signature for {port['name']}")

# Explain the Ports & Adapters pattern
print(f"\n\nPorts & Adapters Pattern Benefits:")
print("=" * 40)
pattern_benefits = [
    "Decouples business logic from external services",
    "Enables easy swapping of implementations",
    "Facilitates testing with mock implementations",
    "Allows multiple adapters for the same port",
    "Keeps core logic stable while external services change"
]

for i, benefit in enumerate(pattern_benefits, 1):
    print(f"{i}. {benefit}")

## 🔄 Dependency Injection & Container

Let's examine the dependency injection system used in the RAG Engine:

In [4]:
# Explore the dependency injection container
def explore_dependency_container():
    """Explore the dependency injection container implementation"""
    try:
        from src.core.bootstrap import get_container
        
        # Get the container instance
        container = get_container()
        
        # Get registered services
        registered_services = list(container._services.keys()) if hasattr(container, '_services') else []
        
        print("Dependency Injection Container Services:")
        print("=" * 50)
        for service in registered_services:
            print(f"  • {service}")
            
        return container, registered_services
        
    except Exception as e:
        print(f"Error exploring container: {e}")
        return None, []

# Also show the bootstrap file content
def show_bootstrap_content():
    """Show the content of the bootstrap file"""
    try:
        bootstrap_path = repo_root / "src" / "core" / "bootstrap.py"
        with open(bootstrap_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        print("\n\nBootstrap File Content (First 50 Lines):")
        print("=" * 50)
        lines = content.split('\n')
        for i, line in enumerate(lines[:50]):
            print(f"{i+1:3d}: {line}")
            
        if len(lines) > 50:
            print(f"... and {len(lines)-50} more lines")
            
    except Exception as e:
        print(f"Error reading bootstrap file: {e}")

# Explore the container
container, services = explore_dependency_container()

# Show bootstrap content
show_bootstrap_content()

## 🧱 Domain-Driven Design Elements

Let's explore the domain-driven design elements in the RAG Engine:

In [5]:
# Explore domain entities
def explore_domain_entities():
    """Explore domain entities in the RAG Engine"""
    try:
        # Import domain entities
        from src.domain import entities
        
        # Get all classes in the entities module
        entity_classes = []
        for name, obj in inspect.getmembers(entities):
            if inspect.isclass(obj) and hasattr(obj, '__module__') and obj.__module__ == entities.__name__:
                entity_classes.append((name, obj))
        
        print("Domain Entities in RAG Engine:")
        print("=" * 50)
        for name, cls in entity_classes:
            print(f"\n{name}:")
            
            # Get class docstring
            if cls.__doc__:
                print(f"  Description: {cls.__doc__.strip()}")
            
            # Get class attributes/methods
            methods = [m for m in dir(cls) if not m.startswith('_') and callable(getattr(cls, m))]
            if methods:
                print(f"  Public Methods: {methods}")
            
            # Try to get constructor signature
            try:
                sig = inspect.signature(cls.__init__)
                print(f"  Constructor: {sig}")
            except ValueError:
                print(f"  Constructor: Could not retrieve signature")
        
        return entity_classes
        
    except Exception as e:
        print(f"Error exploring domain entities: {e}")
        return []

# Also show domain errors
def explore_domain_errors():
    """Explore domain errors in the RAG Engine"""
    try:
        from src.domain import errors
        
        # Get all exception classes in the errors module
        error_classes = []
        for name, obj in inspect.getmembers(errors):
            if inspect.isclass(obj) and issubclass(obj, Exception):
                error_classes.append((name, obj))
        
        print("\n\nDomain Errors in RAG Engine:")
        print("=" * 50)
        for name, cls in error_classes:
            print(f"\n{name}:")
            
            # Get class docstring
            if cls.__doc__:
                print(f"  Description: {cls.__doc__.strip()}")
            
            # Check inheritance
            bases = [b.__name__ for b in cls.__mro__[1:] if b != object]
            print(f"  Inherits from: {', '.join(bases)}")
        
        return error_classes
        
    except Exception as e:
        print(f"Error exploring domain errors: {e}")
        return []

# Explore domain entities and errors
entities = explore_domain_entities()
errors = explore_domain_errors()

## 🔄 Use Cases Implementation

Let's examine the use cases in the RAG Engine:

In [6]:
# Explore use cases
def explore_use_cases():
    """Explore use cases in the RAG Engine"""
    try:
        from src.application.use_cases import ask_hybrid_use_case
        from src.application.use_cases import upload_document_use_case
        
        print("Use Cases in RAG Engine:")
        print("=" * 50)
        
        # Get the AskQuestionHybridUseCase
        for attr_name in dir(ask_hybrid_use_case):
            attr = getattr(ask_hybrid_use_case, attr_name)
            if inspect.isclass(attr) and attr_name.endswith('UseCase'):
                print(f"\n{attr_name}:")
                if attr.__doc__:
                    print(f"  Description: {attr.__doc__.strip()[:100]}...")
                
                # Get constructor signature
                try:
                    sig = inspect.signature(attr.__init__)
                    print(f"  Dependencies: {list(sig.parameters.keys())[1:]}")  # Skip 'self'
                except ValueError:
                    print(f"  Dependencies: Could not retrieve signature")
        
        # Get the UploadDocumentUseCase
        for attr_name in dir(upload_document_use_case):
            attr = getattr(upload_document_use_case, attr_name)
            if inspect.isclass(attr) and attr_name.endswith('UseCase'):
                print(f"\n{attr_name}:")
                if attr.__doc__:
                    print(f"  Description: {attr.__doc__.strip()[:100]}...")
                
                # Get constructor signature
                try:
                    sig = inspect.signature(attr.__init__)
                    print(f"  Dependencies: {list(sig.parameters.keys())[1:]}")  # Skip 'self'
                except ValueError:
                    print(f"  Dependencies: Could not retrieve signature")
        
    except Exception as e:
        print(f"Error exploring use cases: {e}")

# Explore use cases
explore_use_cases()

## 🔧 Adapters Implementation

Let's examine how adapters implement the ports:

In [7]:
# Explore adapters
def explore_adapters():
    """Explore adapters in the RAG Engine"""
    try:
        from src.adapters.llm.openai_llm import OpenAILLM
        from src.adapters.embeddings.openai_embeddings import OpenAIEmbeddings
        from src.adapters.vector.qdrant_adapter import QdrantAdapter
        
        adapters_info = [
            {"name": "OpenAILLM", "class": OpenAILLM, "description": "OpenAI LLM adapter"},
            {"name": "OpenAIEmbeddings", "class": OpenAIEmbeddings, "description": "OpenAI Embeddings adapter"},
            {"name": "QdrantAdapter", "class": QdrantAdapter, "description": "Qdrant vector store adapter"}
        ]
        
        print("Adapters in RAG Engine:")
        print("=" * 50)
        
        for adapter in adapters_info:
            print(f"\n{adapter['name']}: {adapter['description']}")
            
            if adapter['class'].__doc__:
                print(f"  Description: {adapter['class'].__doc__.strip()[:100]}...")
            
            # Get constructor signature
            try:
                sig = inspect.signature(adapter['class'].__init__)
                params = list(sig.parameters.keys())[1:]  # Skip 'self'
                print(f"  Constructor Params: {params}")
            except ValueError:
                print(f"  Constructor: Could not retrieve signature")
            
            # Check what interfaces this adapter implements
            print(f"  Implements: {[base.__name__ for base in adapter['class'].__mro__ if 'Port' in base.__name__ or 'Protocol' in str(base)]}")
        
    except Exception as e:
        print(f"Error exploring adapters: {e}")
        
        # Show the error in more detail
        import traceback
        print(f"Detailed error: {traceback.format_exc()}")

# Explore adapters
explore_adapters()

## 🧩 API Layer & Controllers

Let's examine the API layer and how it connects to use cases:

In [8]:
# Explore API layer
def explore_api_layer():
    """Explore the API layer in the RAG Engine"""
    try:
        # Look at a sample route file
        api_routes_path = repo_root / "src" / "api" / "v1" / "routes_ask.py"
        if api_routes_path.exists():
            with open(api_routes_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            print("API Layer - Ask Route (First 30 Lines):")
            print("=" * 50)
            lines = content.split('\n')
            for i, line in enumerate(lines[:30]):
                print(f"{i+1:3d}: {line}")
            
            if len(lines) > 30:
                print(f"... and {len(lines)-30} more lines")
        
        print(f"\n\nAPI Layer Characteristics:")
        print("=" * 30)
        api_characteristics = [
            "Thin controllers that delegate to use cases",
            "Request validation with Pydantic models",
            "Response formatting",
            "Authentication and tenant extraction",
            "Error handling and mapping to HTTP responses"
        ]
        
        for i, char in enumerate(api_characteristics, 1):
            print(f"{i}. {char}")
        
    except Exception as e:
        print(f"Error exploring API layer: {e}")

# Explore API layer
explore_api_layer()

## 🧪 Testing Architecture

Let's look at how the system is tested to ensure quality:

In [9]:
# Explore testing patterns
def explore_testing_patterns():
    """Explore testing patterns in the RAG Engine"""
    try:
        # Show a sample test file
        test_path = repo_root / "tests" / "unit" / "test_core.py"
        if test_path.exists():
            with open(test_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            print("Testing Patterns - Sample Test File (First 25 Lines):")
            print("=" * 60)
            lines = content.split('\n')
            for i, line in enumerate(lines[:25]):
                print(f"{i+1:3d}: {line}")
            
            if len(lines) > 25:
                print(f"... and {len(lines)-25} more lines")
        
        print(f"\n\nTesting Architecture Patterns:")
        print("=" * 40)
        testing_patterns = [
            "Unit tests for individual components",
            "Integration tests for component interactions",
            "Mock implementations for external dependencies",
            "Pytest for test organization and execution",
            "Type checking with mypy",
            "Code coverage measurement"
        ]
        
        for i, pattern in enumerate(testing_patterns, 1):
            print(f"{i}. {pattern}")
        
    except Exception as e:
        print(f"Error exploring testing patterns: {e}")

# Explore testing patterns
explore_testing_patterns()

## 🔐 Security & Configuration

Let's examine security and configuration patterns:

In [10]:
# Explore security and configuration
def explore_security_config():
    """Explore security and configuration patterns in the RAG Engine"""
    try:
        # Show configuration file
        config_path = repo_root / "src" / "core" / "config.py"
        if config_path.exists():
            with open(config_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            print("Configuration System (First 30 Lines):")
            print("=" * 50)
            lines = content.split('\n')
            for i, line in enumerate(lines[:30]):
                print(f"{i+1:3d}: {line}")
            
            if len(lines) > 30:
                print(f"... and {len(lines)-30} more lines")
        
        print(f"\n\nSecurity & Configuration Patterns:")
        print("=" * 45)
        security_config_patterns = [
            "Pydantic BaseSettings for configuration management",
            "Environment variable loading",
            "Secret management",
            "JWT-based authentication",
            "Tenant isolation mechanisms",
            "Input validation and sanitization",
            "Rate limiting and access controls"
        ]
        
        for i, pattern in enumerate(security_config_patterns, 1):
            print(f"{i}. {pattern}")
        
    except Exception as e:
        print(f"Error exploring security and config: {e}

# Explore security and configuration
explore_security_config()

## 📊 Observability & Monitoring

Let's examine the observability and monitoring components:

In [11]:
# Explore observability
def explore_observability():
    """Explore observability and monitoring in the RAG Engine"""
    try:
        # Show observability file
        obs_path = repo_root / "src" / "core" / "observability.py"
        if obs_path.exists():
            with open(obs_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            print("Observability System (First 25 Lines):")
            print("=" * 50)
            lines = content.split('\n')
            for i, line in enumerate(lines[:25]):
                print(f"{i+1:3d}: {line}")
            
            if len(lines) > 25:
                print(f"... and {len(lines)-25} more lines")
        
        print(f"\n\nObservability Components:")
        print("=" * 30)
        observability_components = [
            "Metrics collection with OpenTelemetry",
            "Structured logging with structlog",
            "Distributed tracing",
            "Error tracking with Sentry",
            "Performance monitoring",
            "Custom business metrics"
        ]
        
        for i, comp in enumerate(observability_components, 1):
            print(f"{i}. {comp}")
        
    except Exception as e:
        print(f"Error exploring observability: {e}")

# Explore observability
explore_observability()

## 🚀 Extensibility & Customization

Let's examine how the system can be extended and customized:

In [12]:
# Explore extensibility patterns
def explore_extensibility():
    """Explore extensibility patterns in the RAG Engine"""
    print("Extensibility & Customization Patterns:")
    print("=" * 50)
    
    extensibility_patterns = [
        {
            "pattern": "Adapter Pattern",
            "description": "Add new implementations of existing ports",
            "example": "Add new LLM provider by implementing LLMPort"
        },
        {
            "pattern": "Plugin Architecture",
            "description": "Register new services in the dependency container",
            "example": "Add custom pre/post-processing services"
        },
        {
            "pattern": "Event System",
            "description": "Hook into system events with custom handlers",
            "example": "Trigger custom actions on document upload completion"
        },
        {
            "pattern": "Configuration Driven",
            "description": "Change behavior through configuration",
            "example": "Switch between different LLM providers via settings"
        },
        {
            "pattern": "New Use Cases",
            "description": "Add new application workflows",
            "example": "Create custom analysis workflows"
        }
    ]
    
    for i, pattern in enumerate(extensibility_patterns, 1):
        print(f"\n{i}. {pattern['pattern']}")
        print(f"   Description: {pattern['description']}")
        print(f"   Example: {pattern['example']}")
    
    print(f"\n\nImplementation Guidelines:")
    print("=" * 30)
    implementation_guidelines = [
        "Always depend on abstractions, not concretions",
        "Follow existing interfaces and patterns",
        "Maintain backward compatibility",
        "Provide comprehensive tests for new components",
        "Document new functionality clearly",
        "Consider performance implications"
    ]
    
    for i, guideline in enumerate(implementation_guidelines, 1):
        print(f"{i}. {guideline}")

# Explore extensibility
explore_extensibility()

## 📚 Key Takeaways

The RAG Engine Mini demonstrates several important architectural principles:

1. **Clean Architecture**: Clear separation of concerns with dependencies pointing inward
2. **Dependency Inversion**: Business rules depend on abstractions, not details
3. **Testability**: Designed for easy mocking and testing of components
4. **Extensibility**: New implementations can be added without changing core logic
5. **Maintainability**: Clear structure makes it easier to understand and modify

The architecture ensures that business logic remains stable while external dependencies can evolve independently.

## 🏁 Conclusion

In this notebook, we've explored the comprehensive architecture of the RAG Engine Mini:

- We examined the Clean Architecture layers and their responsibilities
- We analyzed the Ports & Adapters pattern implementation
- We looked at the dependency injection system and container design
- We explored domain-driven design elements including entities and errors
- We reviewed use cases and how they orchestrate business logic
- We saw how adapters implement ports with concrete implementations
- We observed the thin controller pattern in the API layer
- We understood the testing architecture and patterns
- We reviewed security and configuration patterns
- We explored observability and monitoring components
- We examined extensibility patterns and customization options

This architecture provides a solid foundation for building maintainable, testable, and extensible RAG systems. The clean separation of concerns allows teams to work on different layers independently while maintaining system integrity.